# Modeling : Random forest

In [1]:
from  datetime import datetime, timedelta
# import gc
import numpy as np, pandas as pd
from sklearn.ensemble import RandomForestRegressor
import time	
from sklearn.externals import joblib



/Users/Tommy/opt/anaconda3/envs/python36/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
IS_LOCAL_DEMO = True
# IS_LOCAL_DEMO = False

In [18]:
max_lags = 57
last_day = 1913  
fday = datetime(2016,4, 25) 

In [3]:
data_file = ("train_data.csv.gz" if(IS_LOCAL_DEMO) else "train_data.csv")
dt = pd.read_csv(data_file)

print("loading:",data_file)

loading: train_data.csv.gz


In [6]:
dt["date"] = pd.to_datetime(dt["date"])  


In [7]:
df = dt.copy()

In [8]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):  # [(7, lag_7), (28, lag_28)]
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag) 

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean()) 

    date_features = {
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",

    }
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [9]:
create_fea(df)

In [10]:
df.head()

id  item_id  dept_id  store_id  cat_id  \
0  HOBBIES_1_001_CA_1_validation        0        0         0       0   
1  HOBBIES_1_001_CA_1_validation        0        0         0       0   
2  HOBBIES_1_001_CA_1_validation        0        0         0       0   
3  HOBBIES_1_001_CA_1_validation        0        0         0       0   
4  HOBBIES_1_001_CA_1_validation        0        0         0       0   

   state_id     d  sales       date  wm_yr_wk  ...  sell_price  lag_7  lag_28  \
0         0  1850    0.0 2016-02-21     11604  ...        8.26    NaN     NaN   
1         0  1851    4.0 2016-02-22     11604  ...        8.26    NaN     NaN   
2         0  1852    0.0 2016-02-23     11604  ...        8.26    NaN     NaN   
3         0  1853    1.0 2016-02-24     11604  ...        8.26    NaN     NaN   
4         0  1854    0.0 2016-02-25     11604  ...        8.26    NaN     NaN   

   rmean_7_7  rmean_28_7  rmean_7_28  rmean_28_28  week  quarter  mday  
0        NaN         NaN         NaN          NaN     7        1    21  
1        NaN         NaN         NaN          NaN     8        1    22  
2        NaN         NaN         NaN          NaN     8        1    23  
3        NaN         NaN         NaN          NaN     8        1    24  
4        NaN         NaN         NaN          NaN     8        1    25  

[5 rows x 31 columns]

In [11]:
df.dropna(inplace = True)
df.shape

(90000, 31)

In [12]:
df.columns

Index(['id', 'item_id', 'dept_id', 'store_id', 'cat_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_7', 'lag_28',
       'rmean_7_7', 'rmean_28_7', 'rmean_7_28', 'rmean_28_28', 'week',
       'quarter', 'mday'],
      dtype='object')

In [13]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday", 'event_name_1', 'event_name_2']
train_cols = df.columns[~df.columns.isin(useless_cols)]
print(train_cols)
X_train = df[train_cols]
y_train = df["sales"]

Index(['item_id', 'dept_id', 'store_id', 'cat_id', 'state_id', 'wday', 'month',
       'year', 'event_type_1', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI',
       'sell_price', 'lag_7', 'lag_28', 'rmean_7_7', 'rmean_28_7',
       'rmean_7_28', 'rmean_28_28', 'week', 'quarter', 'mday'],
      dtype='object')


In [14]:
X_train.head(3)

item_id  dept_id  store_id  cat_id  state_id  wday  month  year  \
550000        0        0         0       0         0     1      4  2016   
550001        0        0         0       0         0     2      4  2016   
550002        0        0         0       0         0     3      4  2016   

        event_type_1  event_type_2  ...  sell_price  lag_7  lag_28  rmean_7_7  \
550000             0             0  ...        8.38    1.0     1.0   1.571429   
550001             0             0  ...        8.38    2.0     3.0   1.714286   
550002             0             0  ...        8.38    0.0     1.0   1.714286   

        rmean_28_7  rmean_7_28  rmean_28_28  week  quarter  mday  
550000    1.000000    1.071429     1.035714    15        2    16  
550001    1.428571    1.142857     1.142857    15        2    17  
550002    1.571429    1.142857     1.035714    16        2    18  

[3 rows x 23 columns]

In [15]:
model_rf = RandomForestRegressor()
model_rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [16]:
if(IS_LOCAL_DEMO):
    alphas = [1] # for local demo
else:
    alphas = [1.035, 1.03, 1.025] #[1.015,1.25,1.03] #[1.0,1.15,1.01] #[1.0,1.15,1.01] # for kaggle

weights = [1 / len(alphas)] * len(alphas) 
print(weights)

[1.0]


In [19]:
sub = 0.
cols = [f"F{i}" for i in range(1,29)]

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):  # [(1.035, 0.333), (1.03, 0.333), (1.025, 0.333)]
    te = dt
    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)  
        print(icount, day)
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()  
        create_fea(tst)
        tst = tst.loc[tst.date == day , train_cols] 
        te.loc[te.date == day, "sales"] = alpha * model_rf.predict(tst) 
    
    te_sub = te.loc[te.date >= fday, ["id", "sales"]]  
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]  # [F1,F2,F3,...F28]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()  
    te_sub.fillna(0., inplace = True) 
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)

0 2016-04-25 00:00:00
0 2016-04-26 00:00:00
0 2016-04-27 00:00:00
0 2016-04-28 00:00:00
0 2016-04-29 00:00:00
0 2016-04-30 00:00:00
0 2016-05-01 00:00:00
0 2016-05-02 00:00:00
0 2016-05-03 00:00:00
0 2016-05-04 00:00:00
0 2016-05-05 00:00:00
0 2016-05-06 00:00:00
0 2016-05-07 00:00:00
0 2016-05-08 00:00:00
0 2016-05-09 00:00:00
0 2016-05-10 00:00:00
0 2016-05-11 00:00:00
0 2016-05-12 00:00:00
0 2016-05-13 00:00:00
0 2016-05-14 00:00:00
0 2016-05-15 00:00:00
0 2016-05-16 00:00:00
0 2016-05-17 00:00:00
0 2016-05-18 00:00:00
0 2016-05-19 00:00:00
0 2016-05-20 00:00:00
0 2016-05-21 00:00:00
0 2016-05-22 00:00:00
0 1 1.0


In [20]:
sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)

In [21]:
print(sub.shape)
print(sub.columns)
sub.head(3)

(20000, 29)
Index(['id', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28'],
      dtype='object', name='F')


F                           id    F1    F2    F3    F4    F5    F6    F7  \
0  FOODS_1_001_CA_1_validation  1.32  0.88  0.60  0.66  1.10  1.16  1.26   
1  FOODS_1_001_CA_2_validation  1.54  1.17  1.29  1.21  1.34  2.17  1.86   
2  FOODS_1_001_CA_3_validation  1.05  0.98  1.01  0.71  0.77  2.29  1.48   

F    F8    F9  ...   F19   F20   F21   F22   F23   F24   F25   F26   F27   F28  
0  1.09  0.85  ...  0.83  1.26  1.52  1.05  1.18  1.03  0.70  0.94  1.42  1.32  
1  1.00  0.87  ...  0.93  2.31  1.97  1.37  1.90  1.35  1.64  1.76  2.14  2.38  
2  1.45  1.00  ...  0.88  1.45  1.86  0.93  0.94  0.92  0.98  0.86  1.65  1.39  

[3 rows x 29 columns]

In [23]:
if(IS_LOCAL_DEMO):
    sub.to_csv("submission_rf.csv.gz",compression='gzip',index=False)# for local demo
else:
    sub.to_csv("submission_rf.csv",index=False) # for kaggle  


* ## ramdon forest kaggle result ： 0.21620